In [1]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
import candlestick_patterns_v2 as cp

In [2]:

# Load environment variables from .env file
load_dotenv()
# Access the database URL
db_url = os.getenv('DB_URL')

# Create connect sql database
def create_connection_database(db_url):
    # Database connection string
    db_url = db_url
    # Create an SQLAlchemy engine
    return create_engine(db_url)

# read data from the PostgreSQL table
try:
   with create_connection_database(db_url).connect() as connection:
    table_name = "nifty_top_500_stocks"
    query = f"SELECT * from {table_name};"
    df = pd.read_sql_query(query, con=create_connection_database(db_url))
    print("data pulled from sql table")
except Exception as e:
    print(f"Error: {e}")

data pulled from sql table


In [3]:
def sma(dataframe,column_name,timeperiod):
    dataframe[f"SMA_{timeperiod}"] = dataframe[column_name].rolling(window=timeperiod).mean()
    return dataframe

def ema(dataframe,column_name,timeperiod):
    dataframe[f"EMA_{timeperiod}"] = dataframe[column_name].ewm(span=timeperiod, adjust=False).mean()
    return dataframe

In [4]:
df.SYMBOL.unique()
df_symbol = df[df.SYMBOL == "360ONE"].sort_values(by="DATE")
df_symbol.tail()

,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-20,EQ,1225.35,1260.00,1210.35,1225.35,1223.15,1249.50,1250.26,1270.00,591.5,1452337,1.815797e+09,53665,360ONE
122121,2024-12-23,EQ,1238.05,1280.75,1208.95,1249.50,1225.10,1232.70,1229.79,1280.75,591.5,499836,6.146931e+08,40886,360ONE
123826,2024-12-24,EQ,1230.00,1239.70,1201.20,1232.70,1233.40,1236.35,1226.88,1280.75,591.5,284990,3.496481e+08,20952,360ONE
126157,2024-12-26,EQ,1236.40,1312.10,1214.25,1236.35,1245.00,1256.25,1234.71,1312.10,591.5,1129000,1.393993e+09,45199,360ONE
127273,2024-12-27,EQ,1262.85,1276.95,1221.90,1256.25,1252.35,1250.90,1241.41,1312.10,591.5,827586,1.027370e+09,43131,360ONE


In [5]:
def process_dataframe(df):
    patterns = {
        "Doji": [],
        "Four Price Doji": [],
        "Long Legged Doji": [],
        "Gravestone Doji": [],
        "Dragonfly Doji": [],
        "Hammer": [],
        "Hanging Man": [],
        "Bullish Marubozu": [],
        "Bearish Marubozu": [],
    }

    for _, row in df.iterrows():
        candle = cp.SingleCandlePattern(row)
        patterns["Doji"].append(candle.is_doji())
        patterns["Four Price Doji"].append(candle.is_four_price_doji())
        patterns["Long Legged Doji"].append(candle.is_long_legged_doji())
        patterns["Gravestone Doji"].append(candle.is_gravestone_doji())
        patterns["Dragonfly Doji"].append(candle.is_dragonfly_doji())
        patterns["Hammer"].append(candle.is_hammer())
        patterns["Hanging Man"].append(candle.is_hanging_man())
        patterns["Bullish Marubozu"].append(candle.is_bullish_murubozu())
        patterns["Bearish Marubozu"].append(candle.is_bearish_murubozu())

    # Add results to the DataFrame
    for pattern, results in patterns.items():
        df[pattern] = results

    return df

In [6]:
processed_df = process_dataframe(df_symbol)
display(processed_df.head())

,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,...,SYMBOL,Doji,Four Price Doji,Long Legged Doji,Gravestone Doji,Dragonfly Doji,Hammer,Hanging Man,Bullish Marubozu,Bearish Marubozu
247,2023-12-22,EQ,661.0,664.85,647.05,635.70,655.9,657.90,654.22,1960.0,...,360ONE,True,False,True,False,False,False,True,False,False
246,2023-12-26,EQ,659.0,660.85,648.00,657.90,651.8,651.50,653.57,1960.0,...,360ONE,False,False,False,False,False,False,False,False,False
245,2023-12-27,EQ,657.9,678.50,647.65,651.50,676.0,674.45,670.11,1960.0,...,360ONE,False,False,False,False,False,False,False,False,False
244,2023-12-28,EQ,676.6,715.60,671.45,674.45,709.6,708.80,693.62,1960.0,...,360ONE,False,False,False,False,False,False,False,False,False
243,2023-12-29,EQ,708.0,715.00,687.10,708.80,710.0,709.85,706.13,1960.0,...,360ONE,True,False,True,False,True,False,False,False,False
